In [1]:
import pandas as pd # library for data analsysisi

In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Step 1: Read in data

In [3]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
headersList = ['Postal Code', 'Borough', 'Neighbourhood']
df = pd.DataFrame(tables[0])
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)
df.groupby(by = 'Postal Code', axis = 1, sort = True)
df.reset_index(drop=True, inplace = True)
for i in range(df.shape[0]):
    if(df.at[i, 'Neighbourhood'] == 'Not assigned'):
        df.at[i, 'Neighbourhood'] = df.at[i, 'Borough'] 
print(df.shape)

(103, 3)


Step 2: Add Lat Lon

In [4]:
# initialize your variable to None
df_latlon = pd.read_csv("http://cocl.us/Geospatial_data")
df_latlon.groupby(by = 'Postal Code', axis = 1, sort = True)
for index, row in df_latlon.iterrows():
    pos = str(row['Postal Code'])
    if(pos not in str(df['Postal Code'])):
        continue
    else:
        df.loc[df['Postal Code'] == pos, 'Latitude'] = row['Latitude']
        df.loc[df['Postal Code'] == pos, 'Longitude'] = row['Longitude']

df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [6]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.1.5               |   py36h284efc9_0        11.3 MB  conda-forge
    toolz-0.11.1               |             py_0          46 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        12.1 MB

The following NEW p

In [7]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_data.dropna(inplace = True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
37,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [8]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Cluster those location by K-means using the Latitude and Londtitude

In [13]:
X = toronto_data.values[:, 3:]
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 1], dtype=int32)

In [14]:
toronto_data['Clustered Labels'] = kmeans.labels_
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Clustered Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2
37,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1


In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Clustered Labels']):
    label = '{}, {}'.format(borough, label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto